In [8]:
from pydantic import BaseModel, Field
from typing import Literal
from dotenv import load_dotenv, find_dotenv
import os
import requests

load_dotenv(find_dotenv())
def get_api_key(api_key_name):
    return os.getenv(api_key_name)



In [10]:
class WindInfo(BaseModel):
    speed: float = Field(..., description="Wind speed in km/h")
    direction: int = Field(..., description="Wind direction in radian, e.g., 258'")

class WeatherResponse(BaseModel):
    city: str
    temp: float
    unit: Literal["Celsius", "Fahrenheit", "Kelvin"]
    wind: WindInfo
    
# Example usage
example_json = {
    "city": "Pune",
    "temp": 34.5,
    "unit": "Celsius",
    "wind": {
        "speed": 5.6,
        "direction": "124"
    }
}

weather_data = WeatherResponse(**example_json)
print(weather_data)

city='Pune' temp=34.5 unit='Celsius' wind=WindInfo(speed=5.6, direction=124)


In [14]:
def weahter_report(city_name:str)-> WeatherResponse:
    """ This function generates a weather report for a given city name.
    Args: city name
    Returns: A json containing the weather report.
    """
    base_url = "https://api.openweathermap.org/data/2.5/weather"
    params = {
        "q": city_name,
        "appid": get_api_key("OPENWEATHERMAP_API_KEY"),
        "units": "metric"
    }
    response = requests.get(base_url, params=params)
    
    if response.status_code != 200:
        return f"Error fetching weather: {response.text}"
    data = response.json()
    info = WeatherResponse(
        city = data['name'],
        temp = data['main']['temp'],
        unit = "Celsius",
        wind = WindInfo (
            speed = data["wind"]["speed"],
            direction = data["wind"]["deg"]
        )
    )
    return info


weahter_report('Pune')

WeatherResponse(city='Pune', temp=24.38, unit='Celsius', wind=WindInfo(speed=3.44, direction=254))

In [20]:
cities = ['Pune', 'Mumbai', 'Hyderabad']
for city in cities:
    print("#"*100)
    print("#"," "*10,weahter_report(city), " "*10)
    #print("#"*100)
    

####################################################################################################
#            city='Pune' temp=24.38 unit='Celsius' wind=WindInfo(speed=3.44, direction=254)           
####################################################################################################
#            city='Mumbai' temp=27.44 unit='Celsius' wind=WindInfo(speed=5.16, direction=240)           
####################################################################################################
#            city='Hyderabad' temp=26.67 unit='Celsius' wind=WindInfo(speed=5.34, direction=248)           
